# Model Manager (모델을 통한 조회)
Model Manager는 ModelsCls.objects로 제공

Example)    
`SELECT * FROM app_model;`    
`ModelCls.objects.all()`

`SELECT * FROM app_model ORDER BY id DESC LIMIT 10;`    
`ModelCls.objects.all().order_by('-id')[:10]`

`INSERT INTO app_model (title) VALUES ("New Title");`    
`ModelCls.objects.create(title="New Title")`

In [ ]:
from instagram.models import Post

qs = Post.objects.all() # data를 쿼리할 준비
print(type(qs), qs.query)

# QuerySet
Chaining 지원 -> 조건을 계속해서 덧붙일 수 있다.

In [ ]:
qs = Post.objects.all().filter(message_startswith='첫번째')